In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
from model import load_models, initialize_pipeline
from utils import load_config
from lora import apply_lora_replacement, disable_lora, enable_lora, load_all_lora_weights

torch.set_grad_enabled(False)

/opt/miniconda3/envs/diff_lora/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config()

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LORA_RANK = 128
LORA_ALPHA = 64
ABLATION_STUDY_DIR = os.path.join(config.get("RESULTS_PATH"), "ablation_study")
os.makedirs(ABLATION_STUDY_DIR, exist_ok=True)

PROMPT = "The wolf from the cartoon “Well, Wait!”"
SAVE_DIR = config.get("TRAINING_FOLDER_NAME")
DEVICE

device(type='cpu')

In [3]:
vae, unet, text_encoder, tokenizer, noise_scheduler = load_models()
pipe = initialize_pipeline(vae=vae, unet=unet, text_encoder=text_encoder, tokenizer=tokenizer, noise_scheduler=noise_scheduler, device=DEVICE)

Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

In [4]:
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_before_lora1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_before_lora2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_before_lora3.png"))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
apply_lora_replacement(models=[unet, vae, text_encoder], lora_alpha=LORA_ALPHA, lora_rank=LORA_RANK)

In [ ]:
load_all_lora_weights({"vae": vae, "unet": unet, "text_encoder": text_encoder}, 'weights')

In [7]:
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora3.png"))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Disable LoRA in all models
disable_lora(vae)
disable_lora(unet)
disable_lora(text_encoder)

In [9]:
# Check if LoRA disability is working
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_disable_lora1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_disable_lora2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_disable_lora3.png"))

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Now try disable every part of model and check what part of model is crucial for transfer learning using LoRA
enable_lora(vae)
enable_lora(unet)
enable_lora(text_encoder)

In [ ]:
disable_lora(vae)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_vae_off1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_vae_off2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_vae_off3.png"))
enable_lora(vae)

In [ ]:
disable_lora(unet)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_unet_off1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_unet_off2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_unet_off3.png"))
enable_lora(unet)

In [ ]:
disable_lora(text_encoder)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_text_encoder_off1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_text_encoder_off2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_text_encoder_off3.png"))
enable_lora(text_encoder)

It apears that turning off text encoder causes biggest quality loss although text encoder have less parameters than unet.

Now try disable LoRA in all model's parts and then activate one by one

In [ ]:
# Disable LoRA in all models
disable_lora(vae)
disable_lora(unet)
disable_lora(text_encoder)

In [ ]:
# Use LoRA only in vae
enable_lora(vae)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_vae_only1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_vae_only2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_vae_only3.png"))
disable_lora(vae)

In [ ]:
# Use LoRA only in unet
enable_lora(unet)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_unet_only1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_unet_only2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_unet_only3.png"))
disable_lora(unet)

In [ ]:
# Use LoRA only in text_encoder
enable_lora(text_encoder)
image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_text_encoder_only1.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_text_encoder_only2.png"))

image = pipe(PROMPT).images[0]
image.save(os.path.join(ABLATION_STUDY_DIR, PROMPT + "_lora_text_encoder_only3.png"))
disable_lora(text_encoder)